# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("city_file.csv")
weather_df.head()

,Unnamed: 0,city,lat,temp_max,lon,country,clouds,wind speed,humidity
0,0,Bousso,10.48,92.64,16.71,TD,87,5.61,25
1,1,Hasaki,35.73,59.00,140.83,JP,20,5.82,82
2,2,Longyearbyen,78.22,35.60,15.64,SJ,90,21.92,80
3,3,Khani,41.96,60.80,42.96,GE,17,25.28,51
4,4,Albany,42.60,59.00,-73.97,US,100,1.99,39


Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
locations = weather_df[['lat', 'lon']]
weights = weather_df['humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
city_df = weather_df.loc[(weather_df["lat"] > 70) & (weather_df["lon"] < 80) & (weather_df["clouds"] == 0), :]
city_df = city_df.dropna(how='any')
city_df.reset_index(inplace=True)
del city_df['index']
city_df.head()


,Unnamed: 0,city,lat,temp_max,lon,country,clouds,wind speed,humidity
0,222,Skjervøy,70.03,30.11,20.97,NO,0,16.42,90
1,324,Mehamn,71.04,23.00,27.85,NO,0,11.41,67


In [6]:
hotel_df = city_df.copy()
hotel_df['Hotel Name'] =""

In [7]:
hotellist = []
for i in range(len(hotel_df)):
    lat = hotel_df.at[i, 'lat']
    lon = hotel_df.at[i,'lon']

    params = {
        "location": f"{lat},{lon}",
        "radius": 5000,
        "type" : "lodging",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
hotel_df["Hotel Name"] = hotellist
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

,Unnamed: 0,city,lat,temp_max,lon,country,clouds,wind speed,humidity,Hotel Name
0,222,Skjervøy,70.03,30.11,20.97,NO,0,16.42,90,Skjervøy Byggeservice AS
1,324,Mehamn,71.04,23.00,27.85,NO,0,11.41,67,Mehamn Arctic Hotel AS


In [8]:
hotel_df = hotel_df.rename(columns ={'city': 'City', 'country': 'Country'})
hotel_df

,Unnamed: 0,City,lat,temp_max,lon,Country,clouds,wind speed,humidity,Hotel Name
0,222,Skjervøy,70.03,30.11,20.97,NO,0,16.42,90,Skjervøy Byggeservice AS
1,324,Mehamn,71.04,23.00,27.85,NO,0,11.41,67,Mehamn Arctic Hotel AS


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))